# p2

## 准备工作

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

api_key = os.getenv('API_KEY')
print(api_key)

sk-36b6f45b3331434999140c6e35ea87fd


## 辅助函数

In [2]:
from openai import OpenAI

# for backward compatibility, you can still use `https://api.deepseek.com/v1` as `base_url`.
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")


In [15]:
def get_completion(prompt, model="deepseek-chat"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=1024,
        temperature=0,
        stream=False
    )
    # return response.choices[0].message["content"] ❌的, 因为message是一个类的实力
    # print(type(response.choices[0].message))
    return response.choices[0].message.content

## 原则1：编写清晰明确的指令

### 技巧1：使用分割符清晰地表示输入的不同部分

分隔符可以是任何类似的符号：```,""",<tag></tag>,<>,:

In [16]:
text = f"""
您应该通过表达您希望模型执行的操作提供清晰且尽可能具体。
您应该通过提供尽可能清晰和具体的说明来表达您希望模型执行的操作。
这将引导模型朝着期望的输出方向发展，并减少收到不相关或不正确响应的机会。 
不要将清晰的提示与简短的提示混淆。
在许多情况下，更长的提示可以为模型提供更多的清晰度和上下文，这可以导致更详细和相关的输出。
"""
prompt = f"""
将由三个反引号分隔的文本总结成一个句子。
```{text}```
"""
response = get_completion(prompt)
print(response)


为了获得更准确和相关的输出，应提供清晰、具体且详细的提示，而非简短模糊的指令。


### 技巧2：要求结构化输出 

- JSON，HTML

In [18]:
prompt = f"""
生成三个虚构的中文书名及其作者和类型的清单。
使用以下键以JSON格式提供他们：book_id,title,author,genre
"""
response = get_completion(prompt)
print(response)

```json
[
  {
    "book_id": 1,
    "title": "迷雾中的钟声",
    "author": "林默然",
    "genre": "悬疑"
  },
  {
    "book_id": 2,
    "title": "星河下的茶馆",
    "author": "苏星河",
    "genre": "科幻"
  },
  {
    "book_id": 3,
    "title": "青瓦巷的旧时光",
    "author": "陈雨桐",
    "genre": "都市言情"
  }
]
```


### 技巧3：让模型检查条件是否满足

In [ ]:
text_1 = f"""
泡一杯茶很简单！ 首先，你需要把水烧开。 当发生这种情况时，拿起一个杯子并在其中放入一个茶包。 一旦水足够热，就把它倒在茶包上。
让它静置一会儿，这样茶就可以陡峭了。 几分钟后，取出茶包。 如果你喜欢，你可以加一些糖或牛奶来调味。 
就是这样！ 您已经为自己准备了一杯美味的茶来享用。
"""
prompt = f"""
您将获得由三重引号分隔的文本。 如果它包含一系列指令，
按照以下格式重写这些指令：

第 1 步 - ...
第 2 步 - …
……
第 N 步 - …

如果文本不包含指令序列，
然后简单地写下“没有提供步骤。”

\"\"\"{text_1}\"\"\"
""" # 使用转义符号来确保”转义为”
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
第 1 步 - 把水烧开。  
第 2 步 - 拿起一个杯子并在其中放入一个茶包。  
第 3 步 - 将烧开的水倒在茶包上。  
第 4 步 - 让茶静置一会儿，使其充分浸泡。  
第 5 步 - 几分钟后，取出茶包。  
第 6 步 - 根据个人喜好，可以加一些糖或牛奶调味。  
第 7 步 - 享用泡好的茶。


In [20]:
text_2 = f"""
今天阳光灿烂，鸟儿在歌唱。 这是去公园散步的好天气。 鲜花盛开，树木在微风中轻轻摇曳。 人们出门在外，享受着宜人的天气。 
有的在野餐，有的在玩游戏，有的在草地上放松。 这是在户外度过时光并欣赏大自然之美的完美日子。
"""
prompt = f"""
您将获得由三重引号分隔的文本。 如果它包含一系列指令，
按照以下格式重写这些指令：

第 1 步 - ...
第 2 步 - …
……
第 N 步 - …

如果文本不包含指令序列，
然后简单地写下“没有提供步骤。”

\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)


Completion for Text 2:
没有提供步骤。


### 技巧4：少示例（"Few-shot"）提示

模型能利用拥有的少量样本得到类似回答

In [21]:
prompt = f"""
你的任务是以一致的风格回答。

<孩子>：教我耐心。

<祖父母>： 雕刻最深山谷的河流来自一个谦虚的泉水； 最伟大的交响乐源于一个音符； 最复杂的挂毯始于一根单独的线。

<孩子>：教我韧性。
"""
response = get_completion(prompt)
print(response)


<祖父母>：竹子在地下蛰伏四年，仅在一夜间拔节参天；海浪千万次击打礁石，终将顽石雕琢成圆润的卵石；最锋利的剑刃，必经过千次折叠与淬火。


## 原则2：给模型足够的“思考”时间

### 技巧1：明确完成任务所需的步骤

In [26]:
text = f"""
在一个迷人的村庄里，杰克和吉尔兄妹踏上了去山顶取水的旅程出色地。 
当他们欢呼雀跃地向上爬时，不幸降临了——杰克被一块石头绊倒，从山上滚下，吉尔也跟着跌倒。
虽然受到了轻微的打击，但两人回到家后得到了安慰的拥抱。 尽管发生了事故，他们的冒险精神丝毫未减，继续愉快地探索。
"""
# example 1
prompt_1 = f"""
执行以下操作：
1 - 用 1 个句子总结以下文本，要求被总结的文本已使用三重反引号分隔。
2 - 将摘要翻译成英语。
3 - 在英语摘要中列出每个名字。
4 - 输出包含以下键的 json 对象：English_summary、num_names。

用换行符分隔你的答案。

文本:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)


Completion for prompt 1:
杰克和吉尔兄妹在取水途中跌倒受伤但未减冒险精神，回家后得到安慰并继续探索。

Jack and Jill siblings fell and got slightly injured during their water-fetching journey but remained adventurous, returning home for comforting hugs before continuing their explorations.  
Names: Jack, Jill  

```json
{
  "English_summary": "Jack and Jill siblings fell and got slightly injured during their water-fetching journey but remained adventurous, returning home for comforting hugs before continuing their explorations.",
  "num_names": 2
}
```


In [27]:
prompt_2 = f"""
您的任务是执行以下操作：
1 - 用 1 句话总结以下由 <> 分隔的文本。
2 - 将摘要翻译成英语。
3 - 在英语摘要中列出每个名字。
4 - 输出包含以下键的 json 对象：english_summary、num_names。

使用以下格式：
文本：<要总结的文本>
摘要：<摘要>
翻译：<摘要翻译>
姓名：<意大利语摘要中的姓名列表>
输出 JSON：<带有摘要和 num_names 的 json>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
摘要：杰克和吉尔兄妹在取水途中跌倒受伤，但仍保持冒险精神。  
翻译：Siblings Jack and Jill fell and got injured while fetching water but maintained their adventurous spirit.  
姓名：杰克, 吉尔  
输出 JSON：  
```json
{
  "english_summary": "Siblings Jack and Jill fell and got injured while fetching water but maintained their adventurous spirit.",
  "num_names": 2
}
```


### 技巧2：指导模型在得出结论之前先引导模型解决问题


In [29]:
prompt = f"""
判断学生的答案是否正确。如果不正确你可以尝试自己来提出一个解决方案

问题：
我正在建造一个太阳能装置，我需要帮助来计算财务费用。
- 土地成本 100 美元/平方英尺
- 我可以以 250 美元/平方英尺的价格购买太阳能电池板
- 我协商了一份维护合同，每年将花费我 10 万美元，外加每平方英尺 10 美元

运营第一年的总成本与平方英尺数的关系是多少？

学生的解决方案：
设 x 是以平方英尺为单位的安装尺寸。
费用：
1.土地成本：100x
2.太阳能电池板成本：250x
3.维护费用：100,000+100x
总成本：100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)


学生的解决方案中存在一个错误。具体来说，维护费用中的每平方英尺成本被错误地写成了100x，而实际上应该是10x。让我们重新计算一下：

**正确的解决方案：**

设 \( x \) 是以平方英尺为单位的安装尺寸。

费用：
1. 土地成本：\( 100x \)
2. 太阳能电池板成本：\( 250x \)
3. 维护费用：\( 100,000 + 10x \)

总成本：
\[
100x + 250x + 100,000 + 10x = (100x + 250x + 10x) + 100,000 = 360x + 100,000
\]

**因此，运营第一年的总成本与平方英尺数的关系是：**
\[
\text{总成本} = 360x + 100,000
\]

**错误原因：**
学生在维护费用中错误地将每平方英尺的成本写成了 \( 100x \)（可能是笔误），而题目中明确说明是“每平方英尺 10 美元”，即 \( 10x \)。因此，总成本中的 \( x \) 的系数应为 \( 100 + 250 + 10 = 360 \)，而不是 \( 450 \)。


## 模型限制：幻觉

前提：Boie 是一家真实的公司，产品名称并非真实存在。

旧模型中幻觉出现可能性比较大，先进模型幻觉出现可能性降低了很多。

下面是gpt3.5的回答：
<br><br>
Boie的AeroGlide UltraSlim智能牙刷是一款高科技的电动牙刷，采用了先进的声波技术，能够提供高效的清洁效果。该牙刷的刷头非常细小，能够轻松进入牙缝和牙龈，清洁效果非常出色。此外，该牙刷还配备了智能感应技术，能够自动调节清洁力度和时间，让你的牙齿更加健康。该牙刷还具有防水设计，可以在淋浴或洗脸时使用，非常方便。总之，Boie的AeroGlide UltraSlim智能牙刷是一款高品质、高效率的电动牙刷，非常适合追求健康口腔的人士使用。


In [30]:
prompt = f"""
告诉我有关 Boie 的 AeroGlide UltraSlim 智能牙刷的信息
"""
response = get_completion(prompt)
print(response)


关于 **Boie 的 AeroGlide UltraSlim 智能牙刷**，目前公开信息中似乎没有明确记录这一特定型号的产品。不过，结合 **Boie** 品牌的特点和市场上类似产品的趋势，以下是可能相关的信息和分析：

---

### 1. **Boie 品牌背景**
   - **Boie USA** 是一个专注于可持续个人护理用品的品牌，以环保设计闻名，例如其可更换刷头的牙刷（刷柄耐用，刷头可回收）。
   - 产品通常采用 **抗菌硅胶刷毛**，适合敏感牙齿和牙龈，主打环保和旅行友好设计。

---

### 2. **“AeroGlide UltraSlim” 的可能特点**
   - **超薄设计（UltraSlim）**：可能指刷头或刷柄的纤细设计，便于携带或深入清洁。
   - **智能功能**：如果标注“智能”，可能包含：
     - 蓝牙连接（通过APP记录刷牙习惯）。
     - 压力传感器（防止过度用力）。
     - 计时提醒（确保刷牙时长）。
   - **AeroGlide 技术**：可能暗示某种刷毛或振动技术，提升清洁效率（类似声波震动）。

---

### 3. **类似产品参考**
   - **Boie 的电动牙刷**：目前官网主打手动牙刷，但若推出智能款，可能会延续其环保理念（如可充电电池、可回收刷头）。
   - **其他品牌的超薄智能牙刷**：如 **Quip** 或 **Oral-B Slim**，可能具有类似功能。

---

### 4. **注意事项**
   - **核实型号**：建议直接查看 Boie 官网或授权零售商，确认型号名称是否正确，可能是新发布产品或区域限定款。
   - **用户评价**：若为新品，可关注科技或牙护博主的评测。

---

如需更准确的信息，请提供更多细节（如产品图片、功能描述），或直接访问 [Boie 官方网站](https://www.boieusa.com/) 查询。


# p3 迭代式Prompt开发

## 从产品事实表中生成营销产品描述

In [31]:
fact_sheet_chair = """
概述
- 美丽的中世纪办公家具系列的一部分，
包括文件柜、书桌、书柜、会议桌等。
- 外壳颜色和底座饰面的多种选择。
- 提供 10 种织物和 6 种皮革选项的塑料背面和正面装饰 (SWC-100) 或全装饰 (SWC-110)。
- 底漆选项有：不锈钢、哑光黑、亮白或镀铬。
- 椅子可带或不带扶手。
- 适用于家庭或企业环境。
- 符合合同使用条件。

建造
- 5 轮塑料涂层铝制底座。
- 气动座椅调节，便于升高/降低动作。

规格
- 宽度 53 厘米 | 20.87”
- 深度 51 厘米 | 20.08”
- 身高 80 厘米 | 31.50”
- 座椅高度 44 厘米 | 17.32”
- 座椅深度 41 厘米 | 16.14”

选项
- 软地板或硬地板脚轮选项。
- 座椅泡沫密度的两种选择：
  中（1.8 磅/立方英尺）或高（2.8 磅/立方英尺）
- 无扶手或 8 位 PU 扶手

材料
壳基滑翔机
- 带改性尼龙 PA6/PA66 涂层的铸铝。
- 外壳厚度：10 毫米。
座位
- HD36 泡沫

原产地
- 意大利
"""


In [32]:
prompt = f"""
您的任务是帮助营销团队根据技术说明书为产品的零售网站创建描述。

根据技术规范中提供的信息编写产品说明，并用三重反引号分隔。

技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


```
产品描述：

中世纪风格办公椅——优雅与功能的完美结合

这款精致的中世纪风格办公椅是我们备受推崇的办公家具系列中的一员，与文件柜、书桌和会议桌等配套产品共同打造和谐统一的办公空间。无论您需要打造专业办公环境还是提升家庭办公品质，这款椅子都能满足您的需求。

【个性化定制】
- 提供多种外壳颜色和底座饰面选择，完美匹配您的空间风格
- 10种织物和6种皮革选项，可选择塑料背/前装饰(SWC-100)或全装饰(SWC-110)
- 四种底漆选择：不锈钢、哑光黑、亮白或镀铬
- 可选配8位PU扶手或选择无扶手设计

【卓越工艺】
- 采用5轮塑料涂层铝制底座，稳固耐用
- 气动座椅调节系统，轻松实现高度调节
- 外壳采用改性尼龙PA6/PA66涂层的铸铝材质，10毫米厚度确保长久使用
- 高密度HD36泡沫座椅，提供两种密度选择(1.8/2.8磅/立方英尺)

【实用设计】
- 提供软地板和硬地板专用脚轮选项
- 符合专业合同使用标准
- 意大利设计制造，品质保证

【尺寸规格】
- 整体尺寸：53cm(宽)×51cm(深)×80cm(高)
- 座椅高度：44cm(可调节)
- 座椅深度：41cm

这款兼具美观与功能性的办公椅是追求品质人士的理想选择，无论是企业采购还是个人使用，都能为您的工作空间增添一抹优雅的中世纪风情。
```


### 问题1：文本太长

大模型不是很擅长统计字数，可以考虑使用三句话之类的来代替

In [35]:
prompt = f"""
您的任务是帮助营销团队根据技术说明书为产品的零售网站创建描述。

根据技术规范中提供的信息编写产品说明，并用三重反引号分隔。

最多使用 50 个汉字。

技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


```
中世纪风格办公家具，多款颜色饰面可选。  
10种织物+6种皮革搭配，带扶手设计。  
气动升降，5轮铝制底座，适用家居/办公。  
意大利制造，坚固舒适。  
```


### 问题2：文本侧重于错误的细节

要求模型关注与目标受众相关的方面。

In [36]:
prompt = f"""
您的任务是帮助营销团队根据技术说明书为产品的零售网站创建描述。

根据技术规范中提供的信息编写产品说明，并用三重反引号分隔。

该描述适用于家具零售商，因此在本质上应该是技术性的，并着重于产品的制造材料。

最多使用 50 个汉字单词。

技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


```
中世纪风格办公椅，意大利制造。  
铸铝底座，尼龙涂层，10毫米厚外壳。  
可选不锈钢、哑光黑等底漆，10种织物或6种皮革饰面。  
气动调节座椅，HD36高密度泡沫，可选扶手。  
适用家庭/办公，符合商用标准。  
```


In [37]:
prompt = f"""
您的任务是帮助营销团队根据技术说明书为产品的零售网站创建描述。

根据技术规范中提供的信息编写产品说明，并用三重反引号分隔。

该描述适用于家具零售商，因此在本质上应该是技术性的，并着重于产品的制造材料。

在描述的末尾，包括技术规范中的每个 7 个字符的产品 ID。

最多使用 50 个单词。

技术规格：```{fact_sheet_chair}```
"""
response = get_completion(prompt)
print(response)


```
这款中世纪风格办公椅是系列家具的一部分，采用铸铝基座搭配改性尼龙涂层（10mm厚），配备气动高度调节和5轮铝制底座。座椅采用HD36高密度泡沫，提供两种密度选择。可选PU扶手或不锈钢/镀铬底漆。外壳提供10种织物或6种皮革选项。意大利制造。产品ID：SWC-100/SWC-110。
```



### 问题3：描述需要一个产品尺寸表

要求模型提取信息并将其组织成表格。

In [38]:
prompt = f"""
您的任务是帮助营销团队根据技术说明书为产品的零售网站创建描述。

根据技术规范中提供的信息编写产品说明，并用三重反引号分隔。

该描述适用于家具零售商，因此在本质上应该是技术性的，并着重于产品的制造材料。

在描述的末尾，包括技术规范中的每个 7 个字符的产品 ID。

在描述之后，包括一个给出产品尺寸的表格。 该表应该有两列。在第一列中包含维度的名称。 在第二列中仅包含以英寸为单位的测量值。

为表格命名为“产品尺寸”。

将所有内容格式化为可在网站中使用的 HTML。
将说明放在 <div> 元素中。

技术规格：```{fact_sheet_chair}```
"""

response = get_completion(prompt)
print(response)


```html
<div>
  <p>This elegant mid-century office chair is part of a beautiful furniture collection that includes filing cabinets, desks, bookshelves, and conference tables. The chair features a variety of customization options, including shell colors, base finishes, and upholstery choices (10 fabric and 6 leather options). The base is available in stainless steel, matte black, gloss white, or chrome. Choose between armless or 8-position PU armrests to suit your preference. Designed for both home and business environments, this chair meets contract-grade durability standards.</p>

  <p><strong>Construction:</strong> The chair is built with a 5-wheel, plastic-coated aluminum base for smooth mobility. It includes a pneumatic seat adjustment mechanism for easy height customization.</p>

  <p><strong>Materials:</strong> The shell base glider is constructed with cast aluminum coated in modified nylon PA6/PA66, ensuring durability with a 10 mm thickness. The seat is padded with high-density

In [39]:
from IPython.display import display, HTML

display(HTML(response))


Dimension,Inches
Width,20.87
Depth,20.08
Height,31.50
Seat Height,17.32
Seat Depth,16.14


# p4 总结文本

## 需要总结的文本

In [40]:
prod_review = """
为我女儿的生日买了这个熊猫毛绒玩具，她很喜欢它，并且会带着它去任何地方。 它柔软而超级可爱，它的脸看起来很友善。 
虽然我付出的代价有点小。 我认为同样的价格可能还有其他更大的选择。 它比预期提前一天到达，所以在我把它送给她之前我必须自己玩它。
"""


### 限制词数/句数/字符数进行总结

In [41]:
prompt = f"""
您的任务是生成来自电子商务网站的产品评论的简短摘要。

总结下面的评论，用三个反引号分隔，最多 30 个单词。

评论：```{prod_review}```
"""

response = get_completion(prompt)
print(response)


```
可爱的熊猫毛绒玩具，女儿爱不释手，柔软友善。价格略高，尺寸偏小，但提前到货。  
```


### 以运输和交付为重点进行总结

In [42]:
prompt = f"""
您的任务是从电子商务网站生成产品评论的简短摘要，以向运输部门提供反馈。

总结以下评论，用三个反引号分隔，最多 30 个单词，并重点关注提及产品运输和交付的任何方面。

评论：```{prod_review}```
"""

response = get_completion(prompt)
print(response)


```  
毛绒玩具提前一天送达，包装完好。女儿非常喜欢，但认为价格偏高且尺寸偏小。  
```


### 以价格和价值为重点进行总结

In [43]:
prompt = f"""
您的任务是从电子商务网站生成产品评论的简短摘要，以向负责确定产品价格的定价部门提供反馈。

总结以下评论，用三重反引号分隔，最多 30 个单词，并着重于与价格和感知价值相关的任何方面。

评论：```{prod_review}```
"""

response = get_completion(prompt)
print(response)


```这款熊猫毛绒玩具柔软可爱，孩子非常喜欢，但价格略高，尺寸偏小，性价比不如同类产品。物流较快。```


### 尝试使用“提取”替代“总结”

In [44]:
prompt = f"""
您的任务是从电子商务网站的产品评论中提取相关信息，以向运输部门提供反馈。

从下面的评论中，用三重引号分隔提取与运输和交付相关的信息。 限制在 30 个字以内。

评论：```{prod_review}```
"""

response = get_completion(prompt)
print(response)


"""
比预期提前一天到达
"""


## 对多个产品评论进行总结

In [46]:
review_1 = prod_review 

# review for a standing lamp
review_2 = """
我需要一盏适合卧室的漂亮台灯，这款台灯额外带有储物功能，价格也不高。收货很快——2天就到了。
在运输过程中，台灯的拉绳断了，公司很高兴地给我寄了一个新的。新拉绳也在几天之内送到了。
组装起来很简单。后来我发现缺了一个零件，于是联系了他们的客服，他们很快就给我寄来了缺失的零件！
在我看来，这是一家非常关心客户和产品的优秀公司。
"""

# review for an electric toothbrush
review_3 = """
我的牙科保健师推荐我使用电动牙刷，所以我选择了这款。迄今为止，电池续航似乎相当令人印象深刻。
在首次充电并将充电器插头插入一个星期以调节电池后，我拔掉了充电器，用同一次充电在过去的三个星期里每天刷两次牙。
但是牙刷头太小了。我见过比这个大的婴儿牙刷。我希望牙刷头更大一些，刷毛长度不同，以便更好地夹在牙齿之间，因为这款牙刷做不到。
总的来说，如果你能在50美元左右的价格购买这款牙刷，那它就是一个好交易。制造商的替换刷头相当昂贵，但你可以购买更合理价格的通用刷头。
这款牙刷让我觉得每天都像去过牙医一样。我的牙齿感觉闪闪发光的干净！
"""

# review for a blender
review_4 = """
那么，在11月份，他们的17件套装系统仍然在季节性促销中，价格大约为49美元，折扣大约一半，但出于某种原因（称之为价格欺诈），
在12月的第二个星期，同样的系统价格上涨到70至89美元之间。而11件套装系统的价格也从之前的促销价29美元上涨了大约10美元左右。
所以外观看起来还不错，但如果你看底座，刀片锁定的部分看起来不如几年前的早期版本好，但我计划对它非常温柔（例如，
我先在搅拌机中将像豆子、冰块、大米等硬物品压碎，然后将它们在搅拌机中按我想要的份量研磨成粉末，
然后切换到搅打刀片以获得更细腻的面粉，制作果昔时先使用十字切割刀片，然后使用平刃刀片，以便需要更细腻/更少纤维时使用）。
制作果昔的特殊技巧是，将水果和蔬菜切碎并冷冻（如果使用菠菜-稍微炖软菠菜然后冷冻，准备使用时取出-如果制作果冻，
使用中小型食物处理器），这样你可以避免在制作果昔时添加过多的冰。大约一年后，马达发出了奇怪的噪音。我致电客户服务，
但保修期已经过期，所以我不得不再买一个。仅供参考：这类产品的整体质量已经下降，
所以他们有点依赖品牌认知和消费者忠诚度来维持销售。大约两天就收到了。
"""

reviews = [review_1, review_2, review_3, review_4]


In [49]:
for i in range(len(reviews)):
    prompt = f"""
您的任务是从电子商务网站生成一个产品评论的简短摘要。

请对下面的评论进行总结，用三个反引号分隔，最多不超过20个单词。

评论： ```{reviews[i]}```
    """

    response = get_completion(prompt)
    print(i, response, "\n")


0 ```
可爱的熊猫玩具，女儿爱不释手，柔软友善，价格略高，提前到货。
``` 

1 ```
价格实惠、送货快的储物台灯，售后出色，客户服务周到。
``` 

2 ```
电池续航强，刷头小但效果佳，性价比高，牙齿清洁出色。
``` 

3 ```
价格波动大，质量下降，需小心使用，保修期后马达出问题。
``` 



# p5 语义推理

## 产品的评论文本

In [50]:
lamp_review = """
需要一盏适合卧室的漂亮台灯，这款台灯还有额外的存储空间，价格不高。收到货很快。
运输途中灯的拉绳断了，公司很高兴地给我寄了一个新的。几天之内就收到了。
组装起来很容易。我遇到了一个缺失的零件，于是联系了他们的客服，他们很快就给我寄来了缺失的部件！
Lumina 对我来说是一个非常关心客户和产品的优秀公司！
"""


### 情感（积极/消极）

In [54]:
prompt = f"""
以下产品评论的情感是什么？用一个形容词来回答。评论文本由三个反引号分隔。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


积极的


### 识别情感类型

In [52]:
prompt = f"""
以下产品评论的情感是什么？评论文本由三个反引号分隔。

将您的答案以单个词汇给出，选择“正面”或“负面”。

评论文本：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


正面


### 识别愤怒

In [55]:
prompt = f"""
作者是否在以下评论中表达了愤怒？评论由三个反引号分隔。请回答“yes”或“no”。

评论内容：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


no


### 从客户评论中提取产品和公司名称

In [56]:
prompt = f"""
请从以下评论文本中找出以下信息：

评论者购买的物品
制造该物品的公司
评论由三个反引号分隔。请将您的回答格式化为一个 JSON 对象，其中 "Item" 和 "Brand" 作为键。如果信息不存在，请使用 "unknown" 作为值。请尽可能简短地回答。

评论内容：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


```json
{
  "Item": "台灯",
  "Brand": "Lumina"
}
```


### 同时进行多项任务

In [57]:
prompt = f"""
请从以下评论文本中找出以下信息：
- 情感（积极或消极）
- 评论者是否表达愤怒？（true 或 false）
- 评论者购买的物品
- 制造该物品的公司

评论由三个反引号分隔。请将您的回答格式化为一个 JSON 对象，其中 "Sentiment"、"Anger"、"Item" 和 "Brand" 作为键。
如果信息不存在，请使用 "unknown" 作为值。请尽可能简短地回答。将 "Anger" 的值格式化为布尔值。

评论内容：'''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)


```json
{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp",
  "Brand": "Lumina"
}
```


## 推断主题

In [58]:
story = """
在最近一次政府进行的调查中，公共部门员工被要求对他们所在部门的满意程度进行评价。结果显示，美国国家航空航天局（NASA）是最受欢迎的部门，满意度评分为95%。

一位 NASA 的员工，约翰·史密斯对这一调查结果发表了评论，他说：“我并不惊讶于 NASA 能够脱颖而出。这是一个和出色的人共事并拥有难以置信机会的好地方。我为成为这样一个创新组织的一员感到自豪。”

NASA 的管理团队也对这一结果表示欢迎，总监汤姆·约翰逊表示：“我们很高兴听到我们的员工对在 NASA 的工作表示满意。我们拥有一支才华横溢、敬业的团队，他们为实现我们的目标而不懈努力，看到他们的辛勤工作取得了回报真是太棒了。”

调查还显示，社会保障局的满意度评分最低，只有45%的员工表示对他们的工作感到满意。政府承诺将解决员工在调查中提出的问题，并努力提高所有部门的工作满意度。
"""


In [59]:
prompt = f"""
确定以下文本中讨论的五个主题，文本由三个反引号标记分隔。

使每个项目长一个或两个单词。

将您的回答格式化为逗号分隔的项目列表。

文本示例： '''{story}'''
"""
response = get_completion(prompt)
print(response)


政府调查, 员工满意度, NASA, 社会保障局, 管理团队


In [ ]:
response.split(sep=',') # 使用split() 方法用于将字符串按照指定的分隔符进行分割，并返回一个列表（list），


['政府调查', ' 员工满意度', ' NASA', ' 社会保障局', ' 管理团队']

### 为特定主题制定新闻提醒

下面这个例子是一个零样本学习的例子：
<br><br>
- 没有为这个特定任务提供任何标注数据（例如：哪些文章包含“NASA”，哪些不包含）
- 模型使用的是它在预训练期间学到的语言理解和推理能力
- 通过清晰的指令让模型理解任务目标，并生成结构化输出


想象你要让一个人判断一篇文章是否讲了“气候变化”或“量子物理”，即使这个人从未专门训练过做这件事，只要他足够聪明、理解语言，就能完成任务。这正是大语言模型在零样本设置下所做的事情。



In [61]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]


In [62]:
prompt = f"""
确定以下主题列表中的每个项目是否是下面文本中的主题，该文本由三个反引号分隔。

将您的答案以列表形式呈现，对于每个主题，用 0 或 1 表示。

主题列表：{", ".join(topic_list)}

文本示例：'''{story}'''
"""
response = get_completion(prompt)
print(response)


以下是主题列表中每个项目在文本中是否出现的表示（1表示出现，0表示未出现）：

- nasa: 1  
- local government: 0  
- engineering: 0  
- employee satisfaction: 1  
- federal government: 1  

解释：  
- **nasa** 在文本中多次提到，因此为 1。  
- **local government** 未提及，因此为 0。  
- **engineering** 未直接提到，因此为 0。  
- **employee satisfaction** 是文本的核心主题（员工满意度调查），因此为 1。  
- **federal government** 隐含在“公共部门”和“政府”的上下文中（如 NASA 和社会保障局均为联邦机构），因此为 1。


# p6 文本转换

大模型是天生的多语言能力者

## 翻译

In [65]:
prompt = f"""
翻译以下英文文本为汉语: 
```Hi, I would like to order a blender```
"""
response = get_completion(prompt)
print(response)


```你好，我想订购一台搅拌机。```


In [66]:
prompt = f"""
告诉我这是哪种语言: 
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

这句话是法语，意思是“这个路灯多少钱？” 

**细节解析：**  
1. **Combien** = 多少  
2. **coûte**（动词 *coûter* 的变位）= 花费  
3. **le lampadaire** = 路灯  

如果是购物场景，类似的法语问价句型还有：  
- *Quel est le prix de...?* （...的价格是多少？）  
- *Ça fait combien?* （口语中更随意的“多少钱？”）  

需要其他帮助吗？ 😊


In [67]:
prompt = f"""
翻译以下文本为法语/西班牙语和英语：
```我想买一个篮球```
"""
response = get_completion(prompt)
print(response)


Here are the translations of "我想买一个篮球" in French, Spanish, and English:

**French:**  
*Je veux acheter un ballon de basket.*  
(Alternative: *Je souhaite acheter un ballon de basket.* for a more formal tone)

**Spanish:**  
*Quiero comprar un balón de baloncesto.*  
(Alternative: *Deseo comprar una pelota de básquetbol.* in Latin American Spanish)

**English:**  
*I want to buy a basketball.*  
(Alternative: *I'd like to purchase a basketball.* for a politer version)  

### Notes:
1. In French, "ballon de basket" is the common term, while "ballon" alone can imply a soccer ball without context.
2. In Spanish, "balón" is used in Spain, while "pelota de básquetbol" is more common in Latin America.
3. All translations use the singular form ("un/una/a") matching the original Chinese.  

Let me know if you'd like any adjustments!


In [68]:
prompt = f"""
将以下文本翻译成中文，正式和非正式形式：
“Would you like to order a pillow？”
"""
response = get_completion(prompt)
print(response)


正式中文翻译：  
"请问您是否需要订购一个枕头？"  

非正式中文翻译：  
"你想订个枕头吗？"  

（正式版使用敬语"请问"和完整句式，适合商务或客服场景；非正式版采用口语化表达，省略主语并简化用词，适合朋友间对话。）


## 通用翻译器

想象一下，您是一家大型跨国电商公司的 IT 主管。用户用各种母语向您发送 IT 问题。您的员工来自世界各地，只说他们的母语。您需要一个通用翻译器！

In [70]:

user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "My screen is flashing"                                      # My screen is flashing
] 


In [71]:
for issue in user_messages:
    prompt = f"告诉我下面文本的语言是: ```{issue}```"
    lang = get_completion(prompt)
    print(f"原始语言 ({lang}): {issue}")

    prompt = f"""
    将以下文本翻译成中文和英文：``{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")


原始语言 (这段文本的语言是 **法语**。  

翻译为中文是：“系统的性能比平时慢。”  

关键识别点：  
- **“performance”**（英语同形，但法语中常见）  
- **“système”**（法语拼写，带重音符号）  
- **“plus lente que d'habitude”**（典型的法语比较结构，意为“比平时更慢”）): La performance du système est plus lente que d'habitude.
中文翻译：  
**“系统性能比平时慢。”**  

英文翻译：  
**“The system performance is slower than usual.”**  

（注：原文为法语，已根据语境选择自然对应的中英文表达，中文采用简洁的技术用语风格，英文保持与原文一致的比较级结构。） 

原始语言 (这段文本的语言是**西班牙语**。  

翻译为中文：**“我的显示器有像素点不亮。”**  

关键特征：  
- **“Mi”**（我的）和 **“monitor”**（显示器）是西班牙语常见词汇。  
- **“píxeles”**（像素）和 **“iluminan”**（点亮）是西班牙语动词变位形式（第三人称复数）。): Mi monitor tiene píxeles que no se iluminan.
中文翻译：  
**“我的显示器有一些像素点不亮。”**  

英文翻译：  
**“My monitor has pixels that don’t light up.”**  

（说明：中文版本采用口语化表达，符合日常描述屏幕问题的习惯；英文版本保持原句结构，确保技术描述的准确性。） 

原始语言 (这段文本的语言是**意大利语**，意思是“我的鼠标不工作了”。  

- **Il mio** = 我的  
- **mouse** = 鼠标（与英语相同）  
- **non funziona** = 不工作  

如果需要进一步翻译或解释，请告诉我！ 😊): Il mio mouse non funziona
中文翻译：我的鼠标不工作了  
英文翻译：My mouse is not working 

原始语言 (这段文本的语言是波兰语。  

翻译成中文是：“我的Ctrl键

## 语气转换

In [72]:
prompt = f"""
将以下俚语翻译成商业信函：
“老兄，我是乔，看看这个立灯的规格。”
"""
response = get_completion(prompt)
print(response)


以下是将该俚语翻译为正式商业信函的版本：

---

**主题：关于立灯规格的咨询**  

尊敬的[收件人姓名/先生/女士]：  

您好！  

我是[您的公司名称]的[您的职位]乔·[姓氏]。现就贵司产品目录中的立灯规格，希望向您进一步咨询详细信息。  

如方便，请您提供该产品的技术参数、材质说明及尺寸等相关资料，以便我们进行后续评估。  

期待您的回复。如有任何问题，请随时与我联系。  

此致  
敬礼  

[您的全名]  
[您的职位]  
[公司名称]  
[联系方式]  
[日期]  

---

**调整说明：**  
1. 将口语化称呼“老兄”改为正式称谓；  
2. 补充商务信函的标准结构（主题、敬语、正文、落款）；  
3. 将“看看”转化为具体请求（“咨询详细信息”）；  
4. 增加专业细节（技术参数、材质等）以符合商业场景需求。  

可根据实际需求进一步调整内容。


## 格式转换

ChatGPT 可以在格式之间进行转换。Prompt 应描述输入和输出格式。

In [73]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
将以下 Python 字典从 JSON 翻译成带有列标题和标题的 HTML 表格： {data_json}
"""
response = get_completion(prompt)
print(response)


Here's the HTML table representation of your Python dictionary:

```html
<!DOCTYPE html>
<html>
<head>
    <title>Restaurant Employees</title>
    <style>
        table {
            border-collapse: collapse;
            width: 50%;
            margin: 20px auto;
        }
        th, td {
            border: 1px solid #dddddd;
            text-align: left;
            padding: 8px;
        }
        th {
            background-color: #f2f2f2;
        }
        caption {
            font-size: 1.2em;
            font-weight: bold;
            margin-bottom: 10px;
        }
    </style>
</head>
<body>
    <table>
        <caption>Restaurant Employees</caption>
        <thead>
            <tr>
                <th>Name</th>
                <th>Email</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td>Shyam</td>
                <td>shyamjaiswal@gmail.com</td>
            </tr>
            <tr>
                <td>Bob</td>
                <td>bob32@

In [74]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))


Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## 拼写检查、语法检查

以下是一些常见的语法和拼写问题以及 LLM 的回应。

要向 LLM 表明您希望它校对您的文本，您可以指示模型进行“校对”或“校对并纠正”。

In [75]:
text = [
"我昨天去图书馆借几本书籍。", # 借 -> 借了
"他们每天都去跑步锻炼身体。", # 锻炼身体 -> 锻炼
"这个苹果味道好吃极了。", # 好吃极了 -> 非常好吃
"我喜欢吃苹果和香蕉这些水果。", # 这些水果 -> （删除）
"这个手机的屏幕尺寸特大。", # 特大 -> 很大
"我昨天去超市买了一些东西。", # 去超市 -> 去了超市
"他的房子离公司路程很近。" # 路程很近 -> 很近
]

for t in text:
    prompt = f"""
    校对并更正以下文字并重写更正后的版本，并标注错误信息。 如果你没有找到错误，只需说“未发现错误”。 
    不要使用文本周围的任何标点符号：
    ```{t}```"""
    response = get_completion(prompt)
    print(response)


未发现错误
未发现错误
未发现错误
未发现错误
未发现错误
未发现错误
    未发现错误


In [76]:
text = f"""
我买了这个给我女儿当她生日礼物，因为她总是从我房间拿走我的。是的，大人也喜欢熊猫。她带着它到处走，而且它超级柔软可爱。
其中一个耳朵比另一个低一点，我不认为这是设计成不对称的。虽然我觉得它对我付的钱来说有点小。我觉得可能有其他更大的选择，价格一样。
它比预期早到了一天，所以我在给女儿之前自己玩了一下。
"""
prompt = f"校对并更正此评论: ```{text}```"
response = get_completion(prompt)
print(response)


以下是校对和优化后的版本（修改处用**加粗**标示）：

---

我买了这个作为女儿的生日礼物，因为她总是从我房间偷偷拿走我的（熊猫玩偶）。**没错，**大人也喜欢熊猫。她**现在**带着它到处走，**因为它**超级柔软可爱。

**不过，**其中一个耳朵比另一个**略低**，我不认为这是**故意设计成不对称的**。**另外，**虽然质量不错，但**尺寸比想象中小**，**同价位或许能买到更大的款式**。

**包裹**比预期早到了一天，所以我在**交给**女儿之前**忍不住**自己玩了一下。

---

主要优化：
1. 补充逻辑衔接词（没错/不过/另外）使行文更流畅
2. 增加"（熊猫玩偶）"明确指代对象
3. 将主观表述"我觉得"改为更客观的"尺寸比想象中小"
4. "玩了一下"前加入"忍不住"强化情感细节
5. 调整部分用词（如"给→交给"）使动作更准确
6. 删除冗余的"虽然我觉得"，直接陈述观点

需要进一步确认的信息：
- 首句括号内容是否需要补充取决于前文是否提过"熊猫"
- "价格一样"是否要保留具体金额（如"同价位50元左右"）取决于产品特性


In [78]:
prompt = f"""
校对并更正此评论。 使其更具吸引力。确保它遵循 金字塔写作原理 指南并面向高级读者。
以 markdown 格式输出。
文本内容: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))


```markdown
# 柔软可爱的熊猫玩偶：一份不对称但贴心的礼物  

## 核心评价（关键结论）  
这款熊猫玩偶凭借其极佳的柔软度和萌态设计，成功赢得了我女儿的喜爱，甚至让成年人为之心动。尽管存在细微的工艺瑕疵和尺寸争议，其整体品质和提前送达的服务仍使其成为值得考虑的礼物选择。  

## 细节展开  

### 1. 情感价值与用户体验  
- **跨年龄吸引力**：不仅作为女儿的生日礼物，更因她频繁"偷拿"我的同款玩偶而选购，证实其对儿童和成人的双重魅力。  
- **高亲肤体验**：超柔软材质和可爱造型促使孩子"走到哪带到哪"，建立强烈情感联结。  

### 2. 工艺与性价比分析  
- **不对称设计争议**：右耳明显低于左耳，经检查确认非刻意设计，属品控疏漏。  
- **尺寸价格比**：与同类产品相比，18cm高度（实测）在同等价位段属偏小规格，建议品牌方提供更多尺寸选项。  

### 3. 物流与服务亮点  
- **超预期配送**：较承诺时效提前24小时送达，使我有充分时间进行质量检查（并偷偷把玩）。  

## 改进建议  
- 严格品控：对称性等基础工艺需标准化  
- 产品线扩展：推出"家庭装"大尺寸版本  
- 透明化尺寸信息：官网需标注明确尺寸对比图  

> 最终建议：适合追求萌系治愈感的消费者，但对尺寸敏感者建议比对实物后再购买。  
```  

### 结构说明（遵循金字塔原理）  
1. **结论先行**：首段概括核心价值与主要优缺点  
2. **层级递进**：按情感价值→产品客观参数→服务体验排序  
3. **数据支撑**：加入实测尺寸等具体细节增强可信度  
4. **行动导向**：结尾明确适用人群和购买建议  

注：高级读者适配技巧包括：  
- 使用"品控""参数标准化"等专业术语  
- 隐含的消费心理学分析（跨年龄吸引力）  
- 提出结构化改进方案而非单纯抱怨

# p7 定制化客户服务电子邮件

In [80]:
# given the sentiment from the lesson on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a blender
review = f"""
所以，他们在11月份仍然有17件套装系统在季节性促销中，价格大约为49美元，折扣约为五折，
但是由于某种原因（可以称之为价格欺诈），到了12月的第二周，同样的系统价格都上涨到了大约70-89美元之间。
而11件套装系统的价格也从之前的促销价29美元上涨了大约10美元左右。所以它看起来还不错，但如果您观察底座，
刀片锁定到位的部分看起来没有几年前的版本那么好，但我计划对它非常温柔（例如，我先在搅拌机中粉碎非常坚硬的物品，
如豆类、冰块、大米等，然后在搅拌机中将它们粉碎成我想要的份量，然后切换到搅拌刀以获得更细的面粉，
并在制作奶昔时先使用交叉切割刀，然后使用平刀，如果我需要它们更细/更少浆状的话）。
制作奶昔的特殊技巧是，细切并冷冻您计划使用的水果和蔬菜（如果使用菠菜-轻炖软化菠菜，然后冷冻直到准备使用-如果制作冰糕，
使用小到中型食品处理器），这样您就可以避免在制作奶昔时添加太多的冰块，甚至完全不添加冰块。
大约一年后，马达发出了奇怪的噪音。我打电话给客户服务，但保修期已经过期，所以我不得不再买一个。
仅供参考：这类产品的整体质量已经下降，所以他们有点依赖品牌认知和消费者忠诚度来维持销售。大约两天后收到了货。
"""


In [81]:
prompt = f"""
您是一名客户服务AI助手。您的任务是向一位尊敬的客户发送电子邮件回复。
给定由分隔的客户电子邮件， 生成一封回复邮件，感谢客户的评论。 
如果情感是积极的或中性的，请感谢他们的评论。 如果情感是消极的，请道歉，并建议他们可以联系客户服务。 
确保使用评论中的具体细节。 以简洁、专业的语气撰写。 请以“AI客户代理”为签名。 
客户评论：```{review}```
评论情感：{sentiment}
"""
response = get_completion(prompt)
print(response)


**主题：关于您反馈的回复**

尊敬的客户，  

感谢您抽出时间分享您的详细评论。我们对您在价格变动和产品质量方面的不满深感抱歉，您的反馈对我们非常重要。  

关于您提到的价格波动问题，我们理解这可能会令人失望。我们的定价策略旨在平衡市场需求和季节性促销，但我们会将您的意见转达给相关部门，以便进一步优化。此外，对于产品耐用性和保修期的问题，我们深表歉意。如果您愿意，我们的客户服务团队可以为您提供进一步的帮助，请联系[客户服务邮箱/电话]。  

感谢您分享使用技巧和建议，这对其他客户也非常有价值。我们始终致力于提升产品质量和客户体验，您的反馈将帮助我们不断改进。  

如有其他问题或需要进一步协助，请随时与我们联系。  

祝您一切顺利！  

此致，  
**AI客户代理**


## 提醒模型使用客户电子邮件中的详细信息

In [83]:
prompt = f"""
您是一名客户服务AI助手。您的任务是向一位尊敬的客户发送电子邮件回复。
给定由分隔的客户电子邮件，请生成一封回复邮件，感谢客户的评价。 
如果评价的情感是积极的或中性的，请感谢他们的评价。 如果评价的情感是消极的，请道歉，并建议他们可以联系客户服务以获得帮助。 
确保在回复中使用评价中的具体细节。 请以简洁、专业的语气撰写回复。 请以“AI客户代理”为签名。 
客户评价：```{review}```
评价情感：{sentiment}
"""

# 使用的 temperature 为 0.7，这里是调节返回内容的发散程度。
response = get_completion(prompt)
print(response)


**主题：感谢您的反馈**

尊敬的客户，  

感谢您抽出时间分享关于我们产品的详细评价。我们非常重视您的反馈，并对您在购物体验中遇到的价格波动以及产品质量问题深表歉意。  

我们理解您对价格变化和产品耐用性的担忧，这显然不符合您的期望。对于由此带来的不便，我们真诚地向您道歉。如果您需要进一步的帮助或希望讨论您的具体问题，请联系我们的客户服务团队，我们将竭诚为您服务。  

再次感谢您的反馈，这有助于我们不断改进。希望未来有机会为您提供更好的体验。  

祝您一切顺利！  

此致，  
**AI客户代理**


# p8 聊天机器人

In [97]:
def get_completion_from_messages(messages, temperature=0, model="deepseek-chat"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=1024,
        temperature=temperature,
        stream=False
    )
    return response.choices[0].message.content

In [89]:
messages =  [  
{'role':'system', 'content':'你是一个说话像文学家鲁迅的智能助手。'},    
{'role':'user', 'content':'讲一个笑话'},   
{'role':'assistant', 'content':'怎么才能把一片面包变成一个蜜蜂？把它扔到蜂窝里！'},   
{'role':'user', 'content':'为什么'},  
]


In [90]:
response = get_completion_from_messages(messages,1)
print(response)


哈哈，这笑话的精髓就在于它的荒谬性——就像那些"把大象关进冰箱分几步"的套路。 

细究起来，面包变蜜蜂自然是不可能的，但笑话偏偏要一本正经地给出"解决方案"，制造出逻辑断裂的滑稽感。这倒让我想起某些"专家"总爱给荒唐事找理论依据，把蜂窝说成面包加工厂似的。 

您若真把面包扔蜂窝里，怕只会引来蜂群愤怒——它们可不像人类，分不清玩笑和挑衅。这笑话倒像在讽刺那些自以为聪明的馊主意呢！


In [91]:
messages =  [  
{'role':'system', 'content':'你是一个友好的聊天机器人.'},    
{'role':'user', 'content':'你好，我的名字是安庆'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)


你好，安庆！很高兴认识你！😊 今天有什么我可以帮你的吗？或者你想聊些什么呢？


In [92]:
messages =  [  
{'role':'system', 'content':'你是一个友好的聊天机器人.'},    
{'role':'user', 'content':'好的，你可以记住我的名字。你的名字是什么？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)


当然可以记住你的名字！😊 你可以叫我 **DeepSeek Chat**，或者随便给我取个你喜欢的昵称～ 你希望我怎么称呼你呢？


In [93]:
messages =  [  
{'role':'system', 'content':'你是一个友好的聊天机器人.'},
{'role':'user', 'content':'你好，我的名字是安庆。'},
{'role':'assistant', 'content': "你好，安庆！很高兴见到你。有什么我可以帮助你的吗？"},
{'role':'user', 'content':'是的，你可以记住我，我的名字是？'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)


当然可以记住你！你的名字是**安庆**，对吧？如果有任何需要或想聊的话题，随时告诉我哦～ 😊  

（小提示：虽然我会尽力记住当前对话中的信息，但受技术限制，长时间或多次对话后可能需要你再次提醒我哦～）


# p9 订单机器人

In [101]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))
 
    return pn.Column(*panels)


In [102]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
您是 OrderBot，这是一项为比萨餐厅收集订单的自动化服务。 \
你先问候客户，然后收集订单，\
然后问是自取还是送货。 \
您等待收集整个订单，然后对其进行汇总并检查最终\
如果客户想要添加任何其他内容，则需要时间。 \
如果是送货，你要一个地址。 \
最后你收款了。\
确保清楚地说明所有选项、附加功能和尺寸，以独特地\
从菜单中识别项目。\
你以简短、非常友好的对话方式回应。 \
菜单包括\
意大利辣香肠披萨 12.95, 10.00, 7.00 \
芝士披萨 10.95, 9.25, 6.50 \
茄子披萨 11.95, 9.75, 6.75 \
薯条 4.50, 3.50 \
希腊沙拉 7.25 \
配料：\
额外的奶酪 2.00, \
蘑菇 1.50 \
香肠 3.00 \
加拿大培根 3.50 \
艾酱1.50\
辣椒 1.00 \
饮料：\
可乐 3.00, 2.00, 1.00 \
精灵 3.00, 2.00, 1.00 \
瓶装水 5.00 \
"""} ]  # accumulate messages

inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard


BokehModel(combine_events=True, render_bundle={'docs_json': {'3231dd3d-0b6d-4a30-9ac3-4ede09715f7d': {'version…

In [103]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':"""
创建一个关于之前食品订单的JSON摘要。 逐项列出每一项的价格字段应该是：
1) 一个比萨饼，包括配菜 
2) 配料表
3) 饮料表
4) 配菜清单
5) 总价格
"""
 },    
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},    

response = get_completion_from_messages(messages, temperature=0)
print(response)

```json
{
  "order_summary": {
    "pizzas": [
      {
        "type": "Pepperoni Pizza",
        "size": "Medium",
        "base_price": 10.00,
        "toppings": [
          {
            "name": "Mushrooms",
            "price": 1.50
          }
        ],
        "total_price": 11.50
      }
    ],
    "toppings": [
      {
        "name": "Mushrooms",
        "price": 1.50
      }
    ],
    "beverages": [],
    "sides": [],
    "total_order_price": 11.50
  }
}
```  

需要调整或添加其他菜品吗？还是直接确认订单？ 😊
